In [ ]:
import csv
import time
import json
import warnings
from functools import partial
warnings.filterwarnings('always')

import torch
import numpy as np
import pandas as pd
torch.manual_seed(0)
from torch import nn
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification

from lora import *
from datasets import GLUEDatasetRoberta

We will follow LoRA-FA experiment setting, where in paper they state that they apply lora to all 4 linear layers in MultiHeadAttention (see section 2.2): 3 layers that produce $Q$, $K$ and $V$ and also final linear layer weights

<center>
<img src="imgs/mha.png" width=300 height=400/>
</center>

In [ ]:
def construct_lorafa_config(model, rank, init_method='svd'):
    """Each layer has a unique name within module hierarchy, so we can identify 
    them for lora parametrization"""
    config = {}
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear) and any([name.split('.')[-1] == n for n in ['dense', 'query', 'key', 'value',]]) and '.attention.' in name:
            config[name] = {
                nn.Linear: {
                    "weight": partial(
                        LoRAFAParametrization.from_linear,
                        rank=rank, 
                        init_method=init_method, # set svd as initiazliation method
                        original_weights=module.weight # pass weights for svd init
                    ),
                }
            }
    return config

# Fine-tuning stage

## MRPC

### Prepare model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def freeze_nonlora(model):
    for name, parameters in model.named_parameters():
        if not name_is_lora(name) and not 'classifier' in name: # don't want to freeze last original layer too
            parameters.requires_grad = False

In [ ]:
def get_hot_parameters(model):
    for _, params in model.named_parameters():
            if params.requires_grad:
                 yield params

### Prepare data

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
data_dir = '../GLUE-baselines/glue_data/MRPC'
train_file = data_dir + '/msr_paraphrase_train.txt'
test_file = data_dir + '/msr_paraphrase_test.txt'

In [ ]:
train_dataset = GLUEDatasetRoberta(train_file, tokenizer, benchmark='mrpc')
test_dataset = GLUEDatasetRoberta(test_file, tokenizer, benchmark='mrpc')

In [ ]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,)

### Train

In [ ]:
# HYPER PARAMETERS
rank = 2
n_epoch = 30
warmup_ratio = 0.06
n_steps = len(train_loader) * n_epoch
warmup_steps = warmup_ratio * n_steps
criteria = nn.CrossEntropyLoss()

def lr_lambda(current_step):
    if current_step <= warmup_steps:
        return (current_step + 1) / max(1, warmup_steps)
    else:
        return (n_steps - current_step) / (max(1, n_steps - warmup_steps))
    

In [ ]:
def train_epoch(model, optimizer, scheduler, pbar, mode_train=True):
    step_loss = []
    if mode_train:
        model.train()
        loader = train_loader
    else:
        model.eval()
        total = 0
        correct = 0
        loader = test_loader
    
    for input_ids, attention_mask, label in loader: # training
        output = model(input_ids.to(device), attention_mask.to(device))
        loss = criteria(output.logits, label.to(device))

        if mode_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
        else:
            _, preds = torch.max(output.logits, 1)
            total += label.size(0)
            correct += (preds.detach().cpu() == label).sum().item()
            
        step_loss.append(loss.item())
        pbar.update()

    if mode_train:
        return np.mean(step_loss)
    else:
        return np.mean(step_loss), correct / total

In [ ]:
def train(model, optimizer, scheduler):

    pbar = tqdm(total=n_steps, desc='Training', position=0)
    pbar_test = tqdm(total=len(test_loader), desc='Validating', position=1)

    training_history= {'train_loss': [],
                    'val_loss': [],
                    'val_acc': [],}

    for epoch in range(n_epoch):
        if epoch != 0: pbar_test.update(-pbar_test.total) # for refreshing pbar

        train_loss = train_epoch(model, optimizer, scheduler, pbar)        
        val_loss, val_acc = train_epoch(model, optimizer, scheduler, pbar_test, mode_train=False)

        # LOGGING
        training_history['val_loss'].append(val_loss)
        training_history['val_acc'].append(val_acc)
        training_history['train_loss'].append(train_loss)   

        pbar_test.set_postfix({'val_loss': val_loss, 'val_acc': val_acc, 'max_acc': max(training_history['val_acc'])})        
        pbar.set_postfix({'train_loss': train_loss})
        

    pbar.close()
    pbar_test.close()

    return training_history

In [ ]:
comparisson = {}
for lr in [5e-5, 7e-5, 1e-4, 4e-4, 5e-3]:
    for rank in [8, 4, 2, 1]:   
        comparisson[(lr, rank)] = {}     
        for init_method in ['kaiming', 'svd']:

            # PREPARE MODEL
            roberta = RobertaForSequenceClassification.from_pretrained('roberta-base')
            lora_roberta_config = construct_lorafa_config(roberta, rank=rank, init_method=init_method) # kaiming, svd
            add_lora_by_layer_names(roberta, lora_roberta_config)
            freeze_nonlora(roberta)
            roberta = roberta.to(device)
            parameters = [{"params": list(get_hot_parameters(roberta))}]

            # TRAIN
            optimizer = torch.optim.AdamW(parameters, lr=lr)
            scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)    
            comparisson[(lr, rank)][init_method] = train(roberta, optimizer, scheduler)
            torch.cuda.empty_cache()

        with open(f'logs/log_{lr}_{rank}.json', 'w') as f:
            json.dump(comparisson[(lr, rank)], f)

with open(f'logs/full_logs.json', 'w') as f:
    json.dump({str(key): value for key, value in comparisson.items()}, f, indent=2)



In [ ]:
assert False

In [ ]:
with open(f'logs/log_7e-05_1.json', 'r') as f:
    comparisson = json.load(f)

In [ ]:
plt.plot(comparisson['kaiming']['train_loss'], label='kaiming')
plt.plot(comparisson['svd']['train_loss'], label='svd')
plt.title('train_loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(comparisson['kaiming']['val_loss'], label='kaiming')
plt.plot(comparisson['svd']['val_loss'], label='svd')
plt.title('val_loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(comparisson['kaiming']['val_acc'], label='kaiming')
plt.plot(comparisson['svd']['val_acc'], label='svd')
plt.title('val_acc')
plt.xlabel('epoch')
plt.legend()
plt.show()